# Programming Assignment: Примеры задач анализа текстов

Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/).

Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('SMSSpamCollection.txt', sep = '\t', names = ['target', 'email'])

df.head()

,target,email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама"

In [3]:
emails = df.email
targets = np.where(df.target == 'spam', 1, 0)

Используя `sklearn.feature_extraction.text.CountVectorizer` со стандартными настройками, получите из списка текстов матрицу признаков X.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
cv_est = CountVectorizer()
X = cv_est.fit_transform(emails)

### Question 1

Оцените качество классификации текстов с помощью `LogisticRegression()` с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [14]:
LR = LogisticRegression(random_state = 2)
LR.fit(X, targets)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
cv_scores = cross_val_score(LR, X, targets, scoring = 'f1', cv = 10)

In [20]:
def write_answer_1(x):
    with open("answer1.txt", "w") as fout:
        fout.write(str(x))

In [21]:
write_answer_1(round(float(np.mean(cv_scores)), 1))

### Question 2

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [22]:
X_test = np.array(['FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB', 
                   'FreeMsg: Txt: claim your reward of 3 hours talk time', 
                   'Have you visited the last lecture on physics?', 
                   'Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$', 
                   'Only 99$'])

In [23]:
X_test_matrix = cv_est.transform(X_test)

In [25]:
def write_answer_2(x):
    with open("answer2.txt", "w") as fout:
        fout.write(str(x))

In [41]:
write_answer_2(np.array2string(LR.predict(X_test_matrix))[1:10])

### Question 3

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите **до второго знака** после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [56]:
ngram_mean_scores = []

for i in [(2,2), (3,3), (1,3)]:
    LR_ngram = LogisticRegression(random_state = 2)
    X_ngram = CountVectorizer(ngram_range = i).fit_transform(emails)
    cv_scores_ngram = cross_val_score(LR_ngram, X_ngram, targets, scoring = 'f1', cv = 10)
    ngram_mean_scores.append(str(round(float(np.mean(cv_scores_ngram)), 2)))

In [43]:
def write_answer_3(x):
    with open("answer3.txt", "w") as fout:
        fout.write(str(x))

In [57]:
write_answer_3(" ".join(ngram_mean_scores))

### Question 4

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии `MultinomialNB()`. Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [58]:
from sklearn.naive_bayes import MultinomialNB

In [59]:
ngram_mean_scores_NB = []

for i in [(2,2), (3,3), (1,3)]:
    NB_ngram = MultinomialNB()
    X_ngram = CountVectorizer(ngram_range = i).fit_transform(emails)
    cv_scores_ngram = cross_val_score(NB_ngram, X_ngram, targets, scoring = 'f1', cv = 10)
    ngram_mean_scores_NB.append(str(round(float(np.mean(cv_scores_ngram)), 2)))

In [60]:
def write_answer_4(x):
    with open("answer4.txt", "w") as fout:
        fout.write(str(x))

In [61]:
write_answer_4(" ".join(ngram_mean_scores_NB))

In [62]:
ngram_mean_scores_NB

['0.65', '0.38', '0.89']

### Question 5

Попробуйте использовать в логистической регрессии в качестве признаков Tf-idf из `TfidfVectorizer()` на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf-idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [66]:
tfidf = TfidfVectorizer()

X_tf = tfidf.fit_transform(emails)

In [67]:
cv_scores_tf = cross_val_score(LogisticRegression(random_state = 2), X_tf, targets, scoring = 'f1', cv = 10)

In [69]:
print('CountVestorizer result = {}\nTF-IDF result = {}'.format(round(float(np.mean(cv_scores)), 3), round(float(np.mean(cv_scores_tf)), 3)))

CountVestorizer result = 0.933
TF-IDF result = 0.853


In [71]:
def write_answer_5(x):
    with open("answer5.txt", "w") as fout:
        fout.write(str(x))

In [72]:
write_answer_5(-1)